In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
app_test = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_test.csv.zip', compression='zip')
app_train = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_train.csv.zip', compression='zip')

# preprocessing 

In [3]:
print(app_test.shape, app_train.shape)

(48744, 121) (307511, 122)


In [4]:
df = app_train.append(app_test).reset_index()

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            df[col] = le.transform(df[col])            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [6]:
# one hot encoding
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
df = pd.get_dummies(df, columns= categorical_columns, dummy_na= True)
print(df.dtypes.value_counts())

uint8      147
float64     66
int64       44
dtype: int64


In [7]:
#df.isnull().sum().sort_values(ascending = False).head()

# let's try featuretools (!?)

In [48]:
import featuretools as ft

In [52]:
bureau = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau.csv.zip', compression = 'zip').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau_balance.csv.zip', compression = 'zip').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/POS_CASH_balance.csv.zip', compression = 'zip').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/credit_card_balance.csv.zip', compression = 'zip').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/previous_application.csv.zip', compression = 'zip').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/installments_payments.csv.zip', compression = 'zip').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

In [53]:
es = ft.EntitySet(id = 'clients')

In [54]:
# Entities with a unique index
es = es.entity_from_dataframe(entity_id = 'df', dataframe = df, index = 'SK_ID_CURR')

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_bal', dataframe = bureau_bal, 
                              make_index = True, index = 'bureaubal_index')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

# feature selection

# cross validation

In [ ]:
#from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=27)

#X_train, X_test, y_train, y_test = skf.split(X, y)

In [ ]:
df_train = df[df['TARGET'].notnull()]
y = df_train['TARGET']
df_test = df[df['TARGET'].isnull()]

In [ ]:
del df_train['TARGET']
del df_test['TARGET']

In [ ]:
print(df_train.shape, df_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42)